In [1]:
import QuantLib as ql

In [2]:
ql.Settings.instance().evaluationDate = ql.Date(3, ql.October, 2014)

## Specifying the reference date of a term structure  
In not-too-accurate terms, the reference date of a term structure is where it begins. It can be the
evaluation date, but you might also want it to start on the spot date, for instance.  
We have two possibilities to define a reference date for a curve—even though some particular classes
only allow one of them.  
The first is to define it by means of a (possibly null) offset from the current evaluation date; e.g.,
“two business days after the evaluation date” to define it as the spot date, or “no business days” to
define it as the evaluation date itself. I’ll do it here by building a sample curve over a few swaps.

In [3]:
helpers = [
    ql.SwapRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(rate/100)),
        ql.Period(*tenor), ql.TARGET(),
        ql.Annual,
        ql.Unadjusted,
        ql.Thirty360(ql.Thirty360.BondBasis),
        ql.Euribor6M()
    )
    for tenor, rate in [
        ((2,ql.Years), 0.201),
        ((3,ql.Years), 0.258),
        ((5,ql.Years), 0.464),
        ((10,ql.Years), 1.151),
        ((15,ql.Years), 1.588)
    ]
]

In [4]:
curve1 = ql.PiecewiseFlatForward(0, ql.TARGET(), helpers, ql.Actual360())

In [6]:
dates, rates = zip(*curve1.nodes())

In [11]:
curve1.nodes()

((Date(3,10,2014), 0.0019777694879293093),
 (Date(7,10,2016), 0.0019777694879293093),
 (Date(9,10,2017), 0.0036475517704509294),
 (Date(7,10,2019), 0.007660760701876805),
 (Date(7,10,2024), 0.018414773669420893),
 (Date(8,10,2029), 0.025311634328221498))

The curve built based on these data will be the same as the first, except that we’re specifying its
reference date explicitly as October 3rd (the first passed date).

In [12]:
curve2 = ql.ForwardCurve(dates, rates, ql.Actual360())

In [14]:
print(f"{curve1.referenceDate()} to {curve1.maxDate()}")
print(f"{curve2.referenceDate()} to {curve2.maxDate()}")

October 3rd, 2014 to October 8th, 2029
October 3rd, 2014 to October 8th, 2029


In [15]:
print(curve1.zeroRate(5.0, ql.Continuous))
print(curve2.zeroRate(5.0, ql.Continuous))

0.452196 % Actual/360 continuous compounding
0.452196 % Actual/360 continuous compounding


In [16]:
print(curve1.zeroRate(
    ql.Date(7, ql.September, 2019),
    ql.Actual360(),
    ql.Continuous
))
print(curve2.zeroRate(
    ql.Date(7, ql.September, 2019),
    ql.Actual360(),
    ql.Continuous
))

0.452196 % Actual/360 continuous compounding
0.452196 % Actual/360 continuous compounding
